# **1. 모듈 불러오기**

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

# **2. HTML 정보 가져오기**

In [ ]:
url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EB%8C%80%EA%B5%AC&sort=0&photo=0&field=0&pd=5&ds=2021.09.24&de=2022.09.24&cluster_rank=93&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:1y,a:all&start=1'
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
          'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'lxml')

# **3. 기사 내용 추출하기**

> **3.1 썸네일 추출하기**

In [ ]:
import pandas as pd
start = 1
result_df = pd.DataFrame()

while start < 2000:
    try:
        url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EB%8C%80%EA%B5%AC&sort=0&photo=0&field=0&pd=5&ds=2021.09.24&de=2022.09.24&cluster_rank=93&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:1y,a:all&start=' + str(start)
        headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
          'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')
        img = [ url['src'] for url in soup.find_all('img', attrs={'class': 'thumb api_get'}) ]
        df = pd.DataFrame({'썸네일' : img})
        result_df = pd.concat([result_df, df], ignore_index=True)
        start += 10
    
    except: # 오류발생시 몇 페이지까지 크롤링했는지 page를 확인하기 
        print(start)
        break

In [ ]:
result_df

,썸네일
0,https://search.pstatic.net/common/?src=https%3...
1,https://search.pstatic.net/common/?src=https%3...
2,https://search.pstatic.net/common/?src=https%3...
3,https://search.pstatic.net/common/?src=https%3...
4,https://search.pstatic.net/common/?src=https%3...
...,...
1925,https://search.pstatic.net/common/?src=https%3...
1926,https://search.pstatic.net/common/?src=https%3...
1927,https://search.pstatic.net/common/?src=https%3...
1928,https://search.pstatic.net/common/?src=https%3...


> **3.2 기타 내용 추출하기**
* 보도일자, 언론사, 기사제목, URL, 기사내용

In [ ]:
pressing = []
contenting = []
for start in range(1, 2000, 10):
  url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EB%8C%80%EA%B5%AC&sort=0&photo=0&field=0&pd=5&ds=2021.09.24&de=2022.09.24&cluster_rank=93&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:1y,a:all&start='+str(start)
  headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
          'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
  response = requests.get(url, headers=headers)
  html = response.text
  soup = BeautifulSoup(html, 'html.parser')
  news = soup.select('div.news_wrap.api_ani_send')
  for n in news:
    title = n.select_one('a.news_tit').text
    date = n.select_one('span.info').text
    url = n.select_one('a.news_tit')['href']
    press = n.select_one('a.info.press').text
    content = n.select_one('div.dsc_wrap').text
    press = press.replace('선정', '')
    press = press.replace('언론사', '')
    pressing.append([date, press, title, url, content])

df2 = pd.DataFrame(pressing)

In [ ]:
result = pd.concat([result_df, df2], ignore_index=True, axis=1)

In [ ]:
result.columns = ['썸네일', '보도일자', '언론사', '기사제목', 'URL', '기사내용']

In [ ]:
result

,썸네일,보도일자,언론사,기사제목,URL,기사내용
0,https://search.pstatic.net/common/?src=https%3...,16시간 전,뉴시스,"대구, 23일 1592명 코로나19 확진…사망자 2명",http://www.newsis.com/view/?id=NISX20220924_00...,대구지역 코로나19 확진자 수가 1000명대를 기록했다. 24일 대구시 재난안전대...
1,https://search.pstatic.net/common/?src=https%3...,1일 전,CNB뉴스,"대구시교육청, 전국시도교육감협의회 총회 개최",https://www.cnbnews.com/news/article.html?no=5...,대구시교육청은 지난 22일 제86회 전국시도교육감협의회 총회를 개최했다고 밝혔다....
2,https://search.pstatic.net/common/?src=https%3...,1일 전,연합뉴스,"국민의힘 대구시당, 스마트 물 산업 육성 정책토론회 개최",https://www.yna.co.kr/view/AKR2022092312660005...,국민의힘 대구시당이 주최하는 첫 정책토론회가 23일 물 관련 산업단지인 대구 달성...
3,https://search.pstatic.net/common/?src=https%3...,1일 전,매일신문,수도권 주택 가격 35% 거품…지방도 세종 60%·대구 23% 등,https://news.imaeil.com/page/view/202209230833...,"지방 중에서는 세종(60%), 광주(39%), 부산(31%), 대구(23%) 등은..."
4,https://search.pstatic.net/common/?src=https%3...,1일 전,노컷뉴스,대구시 미래융복합 교통인프라 구축 업무 협약 체결,https://www.nocutnews.co.kr/news/5822205,"대구시는 도로교통공단, 3개 광역자치단체(인천, 광주, 울산), 3개 민간기업(아..."
...,...,...,...,...,...,...
1995,NaN,4일 전,경북일보,"대구변호사회 '애산인권상' 초대 수상자 이용수 할머니, 서울변호사회 '시민인...",http://www.kyongbuk.co.kr/news/articleView.htm...,이용수 할머니는 대구지방변호사회가 2016년 9월 28일 제정한 '애산(愛山) 인...
1996,NaN,2022.09.13.,데일리한국,"대구시, 대구경북연구원 분리해 '자체 연구원 설립' 추진",http://daily.hankooki.com/news/articleView.htm...,대구시청 전경. 사진=대구시 제공 대구시는 경북도가 제안한 '대구경북연구원(이하 ...
1997,NaN,5일 전,베리타스알파,"대구보건대, 예비 언어재활사 선서식 개최",http://www.veritas-a.com/news/articleView.html...,대구보건대 언어치료학과는 16일 본관 3층 대회의실에서 학생과 교수 등 100여명...
1998,NaN,2022.09.16.,쿠키뉴스,대구국제오페라축제 23일 ‘토란도트’로 개막,http://www.kukinews.com/newsView/kuk202209160101,(대구국제오페라축제 제공) 2022.09.16 제19회 대구국제오페라축제가 오는 ...


> **3.3 DataFrame 저장하기**

In [ ]:
result.to_csv("대구시뉴스기사크롤링_ver05.csv", mode='w')

# **4. model input data를 위한 전처리**



> **4.1 특정 문자 제거**
* 대구 관련 기사만 크롤링하다보니, 기사에 대구라는 단어가 굉장히 많이 포함된다.
* '대구'라는 단어는 model에서 학습을 하는데 불필요한 단어이기에 이와 관련된 단어들은 제거해준다.



In [ ]:
train = result[['기사제목']]
train.columns = ['title']
train['title'] = train['title'].str.replace('대구', '')
train['title'] = train['title'].str.replace('광역시', '')
train['title'] = train['title'].str.replace('시', '')
train['title'] = train['title'].str.replace(',', '')
train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,title
0,23일 1592명 코로나19 확진…사망자 2명
1,교육청 전국도교육감협의회 총회 개최
2,국민의힘 당 스마트 물 산업 육성 정책토론회 개최
3,수도권 주택 가격 35% 거품…지방도 세종 60%· 23% 등
4,미래융복합 교통인프라 구축 업무 협약 체결
...,...
1995,변호사회 '애산인권상' 초대 수상자 이용수 할머니 서울변호사회 '민인...
1996,경북연구원 분리해 '자체 연구원 설립' 추진
1997,보건대 예비 언어재활사 선서식 개최
1998,국제오페라축제 23일 ‘토란도트’로 개막




> **4.2 DataFrame 저장하기**



In [ ]:
train.to_csv("대구시뉴스기사크롤링_train_ver07.csv", mode='w')